In [2]:
from transformers import *

nlp_sentence_classif = pipeline('sentiment-analysis')
print(nlp_sentence_classif(
    'Im not going to lie and say I dont watch the show--I do BUT it has a lot and a lot of flaws 1 The Boarding School is perfect The drama is at a minimum Everyone is so nice to each other you know Lets give that a reality check Its IMPOSSIBLE that ANY school is perfect like PCA Free laptops for everyone Big dorm rooms Mini fridges If there was a school like that in real life almost nobody there would be a virgin for one Two everyone there is so rich and its weird how nobody has anything stolen yet 2 Characters really unrealistic First things first who in theyre right minds talk like they do They talk like a perfect teenager would Secondly Logan ReeseMatthew Underwood is an extremely rich boy hot teenage boy My question is why isnt almost ever girl in that school all over him? Hes rich and hot now a days all those girls would be after him even if he was a jerk Also Chase is the most stupidest person ever He is this shy teenager who claims to not be in love with Zoey and over-reacts to everything that involves Zoey She must be BLIND not to see him in love with her  Come on Nick I know you can do better than THAT Please'))


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'NEGATIVE', 'score': 0.9878195524215698}]


In [5]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from transformers import BERT_PRETRAINED_MODEL_ARCHIVE_LIST
print(BERT_PRETRAINED_MODEL_ARCHIVE_LIST)

['bert-base-uncased', 'bert-large-uncased', 'bert-base-cased', 'bert-large-cased', 'bert-base-multilingual-uncased', 'bert-base-multilingual-cased', 'bert-base-chinese', 'bert-base-german-cased', 'bert-large-uncased-whole-word-masking', 'bert-large-cased-whole-word-masking', 'bert-large-uncased-whole-word-masking-finetuned-squad', 'bert-large-cased-whole-word-masking-finetuned-squad', 'bert-base-cased-finetuned-mrpc', 'bert-base-german-dbmdz-cased', 'bert-base-german-dbmdz-uncased', 'cl-tohoku/bert-base-japanese', 'cl-tohoku/bert-base-japanese-whole-word-masking', 'cl-tohoku/bert-base-japanese-char', 'cl-tohoku/bert-base-japanese-char-whole-word-masking', 'TurkuNLP/bert-base-finnish-cased-v1', 'TurkuNLP/bert-base-finnish-uncased-v1', 'wietsedv/bert-base-dutch-cased']


In [7]:
import torch
text = '[CLS] who is Li Jinhong? [SEP] Li Jinhong is a programmer [SEP]'
tokenized_text = tokenizer.tokenize(text)
tokenized_text

['[CLS]',
 'who',
 'is',
 'li',
 'jin',
 '##hong',
 '?',
 '[SEP]',
 'li',
 'jin',
 '##hong',
 'is',
 'a',
 'programmer',
 '[SEP]']

In [8]:
masked_index  = 8
tokenized_text[masked_index] = '[MASK]'
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor

tensor([[  101,  2040,  2003,  5622,  9743, 19991,  1029,   102,   103,  9743,
         19991,  2003,  1037, 20273,   102]])

In [10]:
device = torch.device('cuda:0')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
model.to(device)
segment_ids = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1]
segment_tensors = torch.tensor([segment_ids]).to(device)
tokens_tensor = tokens_tensor.to(device)
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids = segment_tensors)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
predictions = outputs[0]
predicted_index = torch.argmax(predictions[0, masked_index]).item()

In [20]:
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
predicted_token

'li'

In [22]:
for tokerstr in tokenizer.SPECIAL_TOKENS_ATTRIBUTES:
    print(tokerstr)


bos_token
eos_token
unk_token
sep_token
pad_token
cls_token
mask_token
additional_special_tokens


In [29]:
text_list = []
data = open('../dataset/train.txt', 'r', encoding='utf-8')
for i in data.readlines():
    text_list.append(i.split('\t')[0])

In [2]:
import spacy
from spacy import displacy
from pathlib import Path
parser = spacy.load('en_core_web_sm')
doc = 'We focus on the research and education of AI technology'
doc = parser(doc)
svg = displacy.render(doc, style='dep', jupyter=False)
output_path = Path('./dependency_plot.svg')
output_path.open('w', encoding='utf-8').write(svg)

8038